# QUEVO1 - Example project

This notebook shows how you can use QUEVO1 to find the highest entangled quantum circuit using an evolutionary algorithm. The goal is to optimize the M. Wallach entanglement measure for the circuit, which measures the degree of entanglement between the qubits.

The guide will go through the process of initializing the necessary parameters, creating chromosome generations, and simulating them on the qiskit simulator to measure their entanglement with the Wallach entanglement measure. The evolutionary algorithm will then use this measure to evolve the chromosomes towards circuits with higher entanglement. The addition of the M. Wallach entanglement measure as a fitness metric allows the algorithm to search for quantum circuits with a specific structure that maximizes the degree of entanglement between the qubits.

First we import the Quevo library. __Note!__ You need Qiskit for the library to work.

In [1]:
import QUEVO

Then we set up the evovlers parameters. 
- **gates**: The number of quantum gates per circuit.
- **chromosomes**: The number of chromosomes or solutions you want in each generation.
- **generation**: The number of generations you want to evolve before terminating.
- **target_entanglement**: The "target entanglement" in the MW (Meyer-Wallach) entanglement measure is the amount of entanglement that one would like to generate in a particular quantum state. In other words, the MW entanglement measure quantifies how close a given quantum state is to the "desired" entangled state. The MW entanglement measure is based on the concept of "reference states," which are states that have a fixed amount of entanglement that is considered to be the "desired" entanglement for the system. The reference states depend on the number of qubits in the system and the level of entanglement that one wishes to achieve. For example, for a system of two qubits, the reference state with maximal entanglement is the Bell state (|00⟩ + |11⟩)/√2, while the reference state with zero entanglement is a separable state such as |00⟩ or |01⟩.The MW entanglement measure quantifies the deviation of a given quantum state from the reference state with the desired amount of entanglement. It is defined as the difference between the largest eigenvalue of the density matrix of the state and the expected maximum eigenvalue for the reference state. This measure is always non-negative and reaches its maximum value when the state is maximally entangled.!

In [2]:
gates = 5
chromosomes = 20
generations = 10
gate_types = ['cx', 'x', 'h', 'rxx', 'rzz', 'swap', 'z', 'y', 'toffoli'] # possible gates: # h, cx, x, y, z, swap, rzz, rxx, toffoli
target_entanglement = [0.999999]

In [3]:
# Generate initial generation of chromosomes
init_gen = QUEVO.Generation(chromosomes, gates)
init_gen.create_initial_generation(gate_types)

Initial Generation Parents:
[7, 0, 0, 0, 0, 2, 4, 1, 2, 8, 0, 2, 0, 2, 0]
[5, 0, 2, 6, 0, 1, 5, 1, 2, 4, 2, 0, 8, 1, 2]
[7, 1, 1, 3, 0, 2, 4, 0, 2, 7, 1, 2, 0, 2, 1]
[4, 1, 0, 8, 1, 2, 8, 2, 0, 2, 0, 1, 6, 0, 0]
[6, 1, 2, 4, 2, 0, 1, 2, 0, 2, 0, 2, 6, 2, 1]
[1, 1, 2, 0, 0, 2, 4, 1, 0, 6, 2, 2, 4, 1, 2]
[6, 2, 1, 2, 2, 2, 4, 1, 0, 5, 2, 1, 7, 2, 2]
[4, 2, 0, 0, 1, 0, 3, 2, 0, 2, 0, 0, 2, 2, 2]
[8, 0, 1, 5, 2, 1, 5, 0, 2, 7, 2, 1, 5, 2, 0]
[2, 2, 0, 7, 0, 0, 2, 2, 2, 4, 2, 1, 4, 1, 0]
[4, 2, 0, 6, 0, 1, 7, 2, 2, 2, 0, 2, 2, 1, 0]
[3, 2, 1, 1, 0, 1, 5, 1, 0, 8, 0, 1, 5, 2, 1]
[5, 1, 0, 8, 0, 0, 8, 1, 1, 1, 2, 0, 4, 2, 0]
[0, 2, 0, 6, 1, 1, 2, 0, 1, 6, 1, 1, 1, 1, 0]
[8, 1, 2, 2, 2, 1, 7, 2, 2, 1, 2, 0, 3, 2, 0]
[0, 2, 1, 1, 0, 2, 5, 2, 1, 5, 1, 0, 5, 2, 1]
[7, 1, 0, 5, 0, 1, 6, 2, 0, 1, 0, 0, 0, 2, 1]
[5, 2, 0, 3, 2, 0, 7, 1, 0, 1, 2, 2, 5, 0, 2]
[1, 0, 1, 3, 0, 2, 3, 0, 1, 5, 0, 2, 8, 2, 1]
[1, 0, 2, 6, 0, 2, 0, 1, 0, 3, 2, 1, 8, 0, 2]


Then we check all the chromosomes fitness and print the best one found.

In [4]:
init_gen.run_generation(target_entanglement)

print("Fitness for best chromosome: " + str(init_gen.get_best_fitness()) + "\n"
      + "Best chromosome: \n" + str(init_gen.get_best_chromosome()))
print("\n")

Fitness for best chromosome: 0.49999899999999986
Best chromosome: 
[4, 1, 0, 8, 1, 2, 8, 2, 0, 2, 0, 1, 6, 0, 0]




Before we can start the evolution, we need to declare some container values fopr the best performing chromosome.

In [5]:
# Final value placeholders
current_chromosome = init_gen.get_best_chromosome()
best_chromosome = current_chromosome
final_fitness = init_gen.get_best_fitness()

Then we create a loop for the number of generations, and every time find the best chromosome in the generation to be a parent for the next generation.

In [6]:
def run_evolution(number_of_runs, mutation_Prob=0.4):
    
    plot_list = []
    
    for i in range(0, number_of_runs):
        
        #print('Evolution number: ' + str(i+1))
        init_gen = QUEVO.Generation(chromosomes, gates)
        init_gen.create_initial_generation(gate_types)

        init_gen.run_generation(target_entanglement) 
        # Final value placeholders
        current_chromosome = init_gen.get_best_chromosome()
        best_chromosome = current_chromosome
        final_fitness = init_gen.get_best_fitness()

        final_fitness_list = [final_fitness]


      # Mutation loop

        for gen in range(0, generations):
            #print('Running gen nr.' + str(gen +1))
            # Mutate next generation of chromosomes
            init_gen.evolve_into_next_generation(mutation_rate) 

            # Check every Chromosome's fitness
            init_gen.run_generation(target_entanglement)  

            current_fitness = init_gen.get_best_fitness()
            current_chromosome = init_gen.get_best_chromosome()
            print((str(gen + 1)+"\n")+str(current_fitness),end='')
            


            # Check if there is a new_list best chromosome

            if final_fitness > abs(current_fitness):
                final_fitness = current_fitness
                best_chromosome = current_chromosome

            final_fitness_list.append(current_fitness)
            if current_fitness < 0.01:
                break
                
    return final_fitness_list

Last, let's print the result of the best found ciurcuit and visualize it.

In [7]:
print("Best ent. measured: " + str(final_fitness))
print("Best chromosome found: " + str(best_chromosome))
print("\n")
circuit = QUEVO.Circuit(best_chromosome)
circuit.generate_circuit()
circuit.draw()

Best ent. measured: 0.49999899999999986
Best chromosome found: [4, 1, 0, 8, 1, 2, 8, 2, 0, 2, 0, 1, 6, 0, 0]


                            ┌───┐┌───┐┌─┐
q_0: ─■─────────────■────■──┤ H ├┤ Z ├┤M├
      │ZZ(3.7198) ┌─┴─┐  │  └───┘└───┘└╥┘
q_1: ─■───────────┤ X ├──■─────────────╫─
                  └─┬─┘┌─┴─┐           ║ 
q_2: ───────────────■──┤ X ├───────────╫─
                       └───┘           ║ 
c: 1/══════════════════════════════════╩═
                                       0 
